In [1]:
import numpy as np
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import sys
import os
import pandas as pd

In [2]:
"""r0 values to save"""

windows0 = np.arange(0.90, 2.11, 0.05)
windows1 = np.arange(2.2, 4.11, 0.1)
windows = np.append(windows0, windows1)

"""System setup"""

dt = 4*unit.femtoseconds

# Load param and coord files
prmtop = app.AmberPrmtopFile('structures/complex_eq.prmtop')
inpcrd = app.AmberInpcrdFile('structures/complex_eq.inpcrd')

system = prmtop.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, hydrogenMass=1.5*unit.amu, constraints=app.HBonds)  
integrator = mm.LangevinMiddleIntegrator(0.0000*unit.kelvin, 1.0000/unit.picosecond, dt)

simulation = app.Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)


In [3]:
"""RMSD Restraints"""

reference_positions = inpcrd.positions

receptor_atoms = [
    atom.index for atom in simulation.topology.atoms()
    if atom.residue.index in range(0, 172) and atom.name in ('CA', 'C', 'N')
]

ligand_atoms = [
    atom.index for atom in simulation.topology.atoms()
    if atom.residue.index in range(173, 282) and atom.name in ('CA', 'C', 'N')
]

# Add restraining forces for receptor and ligand rmsd
receptor_rmsd_force = mm.CustomCVForce('0.5*k_rec*rmsd^2')
receptor_rmsd_force.addGlobalParameter('k_rec', 100 * unit.kilocalories_per_mole / unit.angstrom**2)
receptor_rmsd_force.addCollectiveVariable('rmsd', mm.RMSDForce(reference_positions, receptor_atoms))
system.addForce(receptor_rmsd_force)

ligand_rmsd_force = mm.CustomCVForce('0.5*k_lig*rmsd^2')
ligand_rmsd_force.addGlobalParameter('k_lig', 100 * unit.kilocalories_per_mole / unit.angstrom**2)
ligand_rmsd_force.addCollectiveVariable('rmsd', mm.RMSDForce(reference_positions, ligand_atoms))
system.addForce(ligand_rmsd_force)

simulation.context.reinitialize(preserveState=True)

for atom in simulation.topology.atoms():
    if atom.index==receptor_atoms[0] and atom.residue.name!='ASN':
        raise ValueError(f'Incorrect residue selection for DCAF16 - residue N1 is missing')
    if atom.index==receptor_atoms[-1] and atom.residue.name!='LEU':
        raise ValueError(f'Incorrect residue selection for DCAF16 - residue L172 is missing')

for atom in simulation.topology.atoms():
    if atom.index==ligand_atoms[0] and atom.residue.name!='SER':
        raise ValueError(f'Incorrect residue selection for BD2 - residue SER174 is missing')
    if atom.index==ligand_atoms[-1] and atom.residue.name!='ASP':
        raise ValueError(f'Incorrect residue selection for BD2 - residue ASP282 is missing')


In [4]:
"""Radial separation CV"""

# 1-indexing from MDAnalysis
rec_interface_res =  [1, 2, 3, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 56, 58, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 153, 154, 155, 156]
lig_interface_res =  [193, 194, 195, 197, 198, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 255, 256, 257, 259, 260, 261, 262, 263, 264, 265]

# Account for OpenMM residue 0-indexing
rec_interface_res = -1 + np.array(rec_interface_res) 
lig_interface_res = -1 + np.array(lig_interface_res)

rec_group = [
    atom.index for atom in simulation.topology.atoms()
    if atom.residue.index in rec_interface_res and atom.name=='CA'
]

lig_group = [
    atom.index for atom in simulation.topology.atoms()
    if atom.residue.index in lig_interface_res and atom.name=='CA'
]

# Define radial distance as collective variable which we will vary
cv = mm.CustomCentroidBondForce(2, "distance(g1,g2)")
cv.addGroup(np.array(rec_group))
cv.addGroup(np.array(lig_group))

# Specify bond groups
bondGroups = [0, 1]
cv.addBond(bondGroups)

r_0 = 1.1 * unit.nanometers #Set initial separation of 11.5 Angstrom

# Define biasing potential
bias_pot = mm.CustomCVForce('0.5 * k_r * (cv-r_0)^2')
bias_pot.addGlobalParameter('k_r', 10 * unit.kilocalories_per_mole / unit.angstrom**2)
bias_pot.addGlobalParameter('r_0', r_0)

bias_pot.addCollectiveVariable('cv', cv)
system.addForce(bias_pot)

simulation.context.reinitialize(preserveState=True)

In [5]:
"""Boresch restraints"""

def obtain_CA_idx(res_idx):

    """Function to obtain the index of the alpha carbon for a given residue index"""
    
    atom_idx = None

    for atom in simulation.topology.atoms():
        if atom.residue.index == res_idx and atom.name=='CA':
            atom_idx = atom.index
    
    return atom_idx
    
# Boresch_residues = [13, 8, 142, 370, 332, 244]

# Define anchor points (1-indexing)
res_b = 8
res_c = 142
res_B = 262
res_C = 222

# Account for OpenMM 0-indexing 
res_b -=1 
res_c -=1
res_B -=1
res_C -=1

# Find atomic indices
idx_b = obtain_CA_idx(res_b)
idx_c = obtain_CA_idx(res_c)
idx_B = obtain_CA_idx(res_B)
idx_C = obtain_CA_idx(res_C)

print('Anchor points:')
for atom in simulation.topology.atoms():
    if atom.index in [idx_b, idx_c, idx_B, idx_C]:
        print(atom)

# Check that we have only selected CA anchor points
all_atoms = rec_group + [idx_b] + [idx_c] + lig_group + [idx_B] + [idx_C]
for atom in simulation.topology.atoms():
    if atom.index in all_atoms and atom.name != 'CA':
        raise ValueError('Select only CA atoms as anchorpoints')
    
print("\nAtomic indices:")
print(f"rec_group = {rec_group}")
print(f"idx_b = {idx_b}")
print(f"idx_c = {idx_c}")
print(f"lig_group = {lig_group}")
print(f"idx_B = {idx_B}")
print(f"idx_C = {idx_C}\n")
    
# Equilibrium values of Boresch dof
theta_A_0 = 1.65
theta_B_0 = 1.62
phi_A_0 = -2.03
phi_B_0 = -0.94
phi_C_0 = 2.07

k_Boresch = 200 * unit.kilocalories_per_mole / unit.radians**2 #Set global force constant

theta_A_pot = mm.CustomCentroidBondForce(3, '0.5 * k_Boresch * (angle(g1,g2,g3)-theta_A_0)^2')
theta_A_pot.addGlobalParameter('theta_A_0', theta_A_0)
theta_A_pot.addGlobalParameter('k_Boresch', k_Boresch)

# Add the particle groups
theta_A_pot.addGroup([idx_b])
theta_A_pot.addGroup(np.array(rec_group))
theta_A_pot.addGroup(np.array(lig_group))

# Add the centroid angle bond
theta_A_pot.addBond([0, 1, 2])

system.addForce(theta_A_pot)

theta_B_pot = mm.CustomCentroidBondForce(3, '0.5 * k_Boresch * (angle(g1,g2,g3)-theta_B_0)^2')
theta_B_pot.addGlobalParameter('theta_B_0', theta_B_0)
theta_B_pot.addGlobalParameter('k_Boresch', k_Boresch)

# Add the particle groups
theta_B_pot.addGroup(np.array(rec_group))
theta_B_pot.addGroup(np.array(lig_group))
theta_B_pot.addGroup([idx_B])

# Add the centroid angle bond
theta_B_pot.addBond([0, 1, 2])

system.addForce(theta_B_pot)

phi_A_pot = mm.CustomCentroidBondForce(4, "0.5*k_Boresch*min(dtheta, 2*pi-dtheta)^2; dtheta = abs(dihedral(g1,g2,g3,g4)-phi_A_0); pi = 3.1415926535")
phi_A_pot.addGlobalParameter('phi_A_0', phi_A_0)
phi_A_pot.addGlobalParameter('k_Boresch', k_Boresch)

# Add the particle groups
phi_A_pot.addGroup([idx_c])
phi_A_pot.addGroup([idx_b])
phi_A_pot.addGroup(np.array(rec_group))
phi_A_pot.addGroup(np.array(lig_group))

# Add the centroid angle bond
phi_A_pot.addBond([0, 1, 2, 3])

system.addForce(phi_A_pot)

phi_B_pot = mm.CustomCentroidBondForce(4, "0.5*k_Boresch*min(dtheta, 2*pi-dtheta)^2; dtheta = abs(dihedral(g1,g2,g3,g4)-phi_B_0); pi = 3.1415926535")
phi_B_pot.addGlobalParameter('phi_B_0', phi_B_0)
phi_B_pot.addGlobalParameter('k_Boresch', k_Boresch)

# Add the particle groups
phi_B_pot.addGroup([idx_b])
phi_B_pot.addGroup(np.array(rec_group))
phi_B_pot.addGroup(np.array(lig_group))
phi_B_pot.addGroup([idx_B])

# Add the centroid angle bond
phi_B_pot.addBond([0, 1, 2, 3])

system.addForce(phi_B_pot)

phi_C_pot = mm.CustomCentroidBondForce(4, "0.5*k_Boresch*min(dtheta, 2*pi-dtheta)^2; dtheta = abs(dihedral(g1,g2,g3,g4)-phi_C_0); pi = 3.1415926535")
phi_C_pot.addGlobalParameter('phi_C_0', phi_C_0)
phi_C_pot.addGlobalParameter('k_Boresch', k_Boresch)

# Add the particle groups
phi_C_pot.addGroup(np.array(rec_group))
phi_C_pot.addGroup(np.array(lig_group))
phi_C_pot.addGroup([idx_B])
phi_C_pot.addGroup([idx_C])

# Add the centroid angle bond
phi_C_pot.addBond([0, 1, 2, 3])

system.addForce(phi_C_pot)

simulation.context.reinitialize(preserveState=True)

Anchor points:
<Atom 107 (CA) of chain 0 residue 7 (LEU)>
<Atom 2233 (CA) of chain 0 residue 141 (VAL)>
<Atom 3506 (CA) of chain 0 residue 221 (ASP)>
<Atom 4147 (CA) of chain 0 residue 261 (VAL)>

Atomic indices:
rec_group = [4, 18, 37, 56, 96, 107, 136, 160, 177, 193, 215, 226, 245, 264, 286, 307, 318, 332, 346, 920, 941, 2068, 2084, 2095, 2102, 2112, 2123, 2133, 2140, 2164, 2183, 2197, 2414, 2428, 2442, 2463]
idx_b = 107
idx_c = 2233
lig_group = [3045, 3055, 3065, 3096, 3128, 3211, 3227, 3239, 3255, 3270, 3280, 3299, 3306, 3325, 3342, 3354, 4040, 4061, 4083, 4103, 4115, 4132, 4147, 4163, 4179, 4189]
idx_B = 4147
idx_C = 3506



In [7]:
print(lig_group)

[3045, 3055, 3065, 3096, 3128, 3211, 3227, 3239, 3255, 3270, 3280, 3299, 3306, 3325, 3342, 3354, 4040, 4061, 4083, 4103, 4115, 4132, 4147, 4163, 4179, 4189]


In [10]:
lengths = []

for word in ['red', 'green', 'blue']:
    lengths.append(len(word))

print(lengths)

[3, 5, 4]


In [ ]:
lengths = ''

for word in ['red', 'green', 'blue']:
    print(lengths)
    lengths = lengths + word + ' '

print(lengths)


red 
red green 
red green blue 


In [4]:
import math

angle_rad = math.pi/2 # radians
angle_deg = math.degrees(angle_rad)
print(angle_deg)

90.0


In [6]:
acronym = ""

for colour in ['red', 'green', 'blue']:
    first_letter = colour[0]
    acronym = acronym + first_letter.upper()

acronym

'RGB'